# Topic Modeling on an Information Retrieval Textbook

In [70]:
%%time
# Get the path to all text file in dir. Including the files in subdirectories
import os

dir = 'corpus'
file_list = [os.path.join(path, file) for path, _, files in os.walk(dir) 
             for file in files if file.endswith('.txt')]

CPU times: user 1.56 ms, sys: 14.9 ms, total: 16.5 ms
Wall time: 172 ms


In [73]:
%%time
# One pass on all files to create a vocabulary
from nltk.corpus import stopwords
# If you get LookupError use nltk.download() to get the stopword list
# from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

import re
import os

rm_non_alnum = re.compile('[^a-zA-Z]')
# s = PorterStemmer()
lemmatizer = WordNetLemmatizer()

stop_set = set(stopwords.words('english'))
stop_set.update(['et', 'al', 'chapter'])
corp = []

for file in file_list:
    with open(file, 'r', encoding='latin-1') as f:
        corp += [[lemmatizer.lemmatize(word) for line in f 
                  for word in map(lambda x: rm_non_alnum.sub('', x), line.lower().strip().split()) 
                  if len(word) > 0 and word not in stop_set]]

CPU times: user 2.69 s, sys: 185 ms, total: 2.87 s
Wall time: 5.38 s


In [84]:
%%time
from gensim import corpora

dictionary = corpora.Dictionary(corp)

dictionary.filter_extremes(no_below=2)

corpus = [dictionary.doc2bow(text) for text in corp]

CPU times: user 99.6 ms, sys: 8.37 ms, total: 108 ms
Wall time: 101 ms


In [81]:
from gensim.models import LdaModel
%time lda_model = LdaModel(corpus, num_topics=10, id2word=dictionary, passes=4)
lda_model.print_topics(-1) 

CPU times: user 2.93 s, sys: 15.6 ms, total: 2.95 s
Wall time: 2.94 s


[(0,
  '0.025*"document" + 0.011*"clustering" + 0.011*"cluster" + 0.009*"user" + 0.009*"search" + 0.008*"similarity" + 0.007*"page" + 0.007*"measure" + 0.007*"two" + 0.006*"information"'),
 (1,
  '0.027*"document" + 0.019*"term" + 0.017*"query" + 0.015*"zone" + 0.010*"model" + 0.009*"score" + 0.008*"relevance" + 0.007*"weighting" + 0.007*"boolean" + 0.007*"probabilistic"'),
 (2,
  '0.021*"document" + 0.016*"term" + 0.011*"query" + 0.008*"word" + 0.007*"one" + 0.007*"clustering" + 0.006*"two" + 0.006*"index" + 0.006*"cluster" + 0.005*"figure"'),
 (3,
  '0.020*"document" + 0.013*"classification" + 0.012*"model" + 0.012*"class" + 0.011*"classifier" + 0.009*"query" + 0.008*"method" + 0.008*"set" + 0.008*"text" + 0.008*"data"'),
 (4,
  '0.022*"document" + 0.013*"term" + 0.008*"feature" + 0.008*"query" + 0.008*"set" + 0.008*"class" + 0.008*"vector" + 0.007*"example" + 0.007*"precision" + 0.007*"value"'),
 (5,
  '0.016*"document" + 0.011*"clustering" + 0.010*"cluster" + 0.010*"query" + 0.010*

In [82]:
from gensim.models import TfidfModel, LdaModel
%time tfidf_model = TfidfModel(corpus, id2word=dictionary)
%time tfidf_lda = LdaModel(tfidf_model[corpus], id2word=dictionary, num_topics=10, passes=4)
lda_model.print_topics(-1) 

CPU times: user 31.6 ms, sys: 4.28 ms, total: 35.9 ms
Wall time: 32.5 ms
CPU times: user 2.85 s, sys: 14.5 ms, total: 2.87 s
Wall time: 2.87 s


[(0,
  '0.025*"document" + 0.011*"clustering" + 0.011*"cluster" + 0.009*"user" + 0.009*"search" + 0.008*"similarity" + 0.007*"page" + 0.007*"measure" + 0.007*"two" + 0.006*"information"'),
 (1,
  '0.027*"document" + 0.019*"term" + 0.017*"query" + 0.015*"zone" + 0.010*"model" + 0.009*"score" + 0.008*"relevance" + 0.007*"weighting" + 0.007*"boolean" + 0.007*"probabilistic"'),
 (2,
  '0.021*"document" + 0.016*"term" + 0.011*"query" + 0.008*"word" + 0.007*"one" + 0.007*"clustering" + 0.006*"two" + 0.006*"index" + 0.006*"cluster" + 0.005*"figure"'),
 (3,
  '0.020*"document" + 0.013*"classification" + 0.012*"model" + 0.012*"class" + 0.011*"classifier" + 0.009*"query" + 0.008*"method" + 0.008*"set" + 0.008*"text" + 0.008*"data"'),
 (4,
  '0.022*"document" + 0.013*"term" + 0.008*"feature" + 0.008*"query" + 0.008*"set" + 0.008*"class" + 0.008*"vector" + 0.007*"example" + 0.007*"precision" + 0.007*"value"'),
 (5,
  '0.016*"document" + 0.011*"clustering" + 0.010*"cluster" + 0.010*"query" + 0.010*

In [83]:
from gensim.models import TfidfModel, LsiModel
%time tfidf_model = TfidfModel(corpus, id2word=dictionary)
%time lsi_model = LsiModel(tfidf_model[corpus], id2word=dictionary, num_topics=10)
lsi_model.print_topics(-1)

CPU times: user 33.2 ms, sys: 4.88 ms, total: 38.1 ms
Wall time: 34.4 ms
CPU times: user 613 ms, sys: 26.8 ms, total: 640 ms
Wall time: 335 ms


[(0,
  '0.171*"clustering" + 0.161*"document" + 0.158*"cluster" + 0.150*"model" + 0.146*"query" + 0.137*"classification" + 0.126*"class" + 0.124*"relevance" + 0.122*"classifier" + 0.117*"term"'),
 (1,
  '0.556*"clustering" + 0.464*"cluster" + 0.222*"similarity" + 0.164*"singlelink" + 0.152*"centroid" + 0.141*"exercise" + 0.129*"merge" + 0.120*"completelink" + 0.116*"hac" + 0.101*"hierarchical"'),
 (2,
  '0.975*"exercise" + -0.098*"clustering" + -0.082*"cluster" + 0.046*"classifier" + 0.043*"classification" + 0.041*"class" + 0.032*"training" + -0.032*"similarity" + 0.031*"margin" + -0.028*"posting"'),
 (3,
  '-0.306*"classifier" + -0.293*"classification" + -0.276*"class" + 0.257*"posting" + -0.189*"training" + 0.182*"list" + -0.178*"feature" + 0.164*"index" + 0.143*"exercise" + -0.136*"learning"'),
 (4,
  '-0.294*"feedback" + 0.281*"posting" + -0.273*"relevance" + -0.241*"model" + 0.184*"list" + -0.168*"language" + 0.157*"classifier" + -0.152*"probability" + 0.141*"index" + -0.139*"mode

# Topic Modeling on 400+ Research Papers

In [7]:
%%time
# Get the path to all text file in dir. Including the files in subdirectories
import os

dir = 'Good'
file_list = [os.path.join(path, file) for path, _, files in os.walk(dir) 
             for file in files if file.endswith('.txt')]

CPU times: user 2.49 ms, sys: 23.4 ms, total: 25.9 ms
Wall time: 195 ms


In [8]:
%%time
# One pass on all files to create a vocabulary
from nltk.corpus import stopwords
# If you get LookupError use nltk.download() to get the stopword list
# from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

import re
import os

rm_non_alnum = re.compile('[^a-zA-Z]')
s = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stop_set = set(stopwords.words('english'))
# stop_set.update(['et', 'al', 'chapter'])
corp = []

for file in file_list:
    with open(file, 'r', encoding='latin-1') as f:
        corp += [[lemmatizer.lemmatize(word) for line in f 
                  for word in map(lambda x: rm_non_alnum.sub('', x), line.lower().strip().split()) 
                  if len(word) > 0 and word not in stop_set]]

CPU times: user 11.7 s, sys: 591 ms, total: 12.3 s
Wall time: 17.8 s


In [9]:
%%time
from gensim import corpora

dictionary = corpora.Dictionary(corp)

dictionary.filter_extremes(no_below=10)

corpus = [dictionary.doc2bow(text) for text in corp]

CPU times: user 1.4 s, sys: 28.8 ms, total: 1.43 s
Wall time: 1.43 s


In [10]:
from gensim.models import LdaModel
%time lda_model = LdaModel(corpus, num_topics=10, id2word=dictionary, passes=4)
lda_model.print_topics(-1) 

CPU times: user 1min 2s, sys: 2.2 s, total: 1min 5s
Wall time: 16.5 s


[(0,
  '0.010*"writing" + 0.007*"text" + 0.007*"item" + 0.007*"word" + 0.006*"discourse" + 0.005*"language" + 0.005*"score" + 0.005*"collaborative" + 0.004*"essay" + 0.004*"sentence"'),
 (1,
  '0.027*"network" + 0.012*"cluster" + 0.010*"forum" + 0.007*"week" + 0.007*"mooc" + 0.005*"participant" + 0.005*"tie" + 0.005*"actor" + 0.005*"moocs" + 0.004*"node"'),
 (2,
  '0.007*"emotion" + 0.007*"grade" + 0.006*"predictive" + 0.006*"score" + 0.004*"assignment" + 0.004*"day" + 0.004*"instructor" + 0.004*"week" + 0.004*"visualization" + 0.004*"prediction"'),
 (3,
  '0.007*"la" + 0.006*"institution" + 0.005*"privacy" + 0.005*"policy" + 0.005*"reflection" + 0.004*"collaborative" + 0.004*"workshop" + 0.003*"video" + 0.003*"pedagogical" + 0.003*"institutional"'),
 (4,
  '0.008*"dialogue" + 0.006*"participant" + 0.005*"message" + 0.004*"cognitive" + 0.004*"reading" + 0.004*"network" + 0.004*"visualisation" + 0.004*"eeg" + 0.004*"utterance" + 0.004*"gesture"'),
 (5,
  '0.005*"condition" + 0.004*"lect

In [11]:
from gensim.models import TfidfModel, LdaModel
%time tfidf_model = TfidfModel(corpus, id2word=dictionary)
%time tfidf_lda = LdaModel(tfidf_model[corpus], id2word=dictionary, num_topics=10, passes=4)
lda_model.print_topics(-1) 

CPU times: user 325 ms, sys: 21.9 ms, total: 347 ms
Wall time: 85.9 ms
CPU times: user 56.3 s, sys: 1.8 s, total: 58.1 s
Wall time: 19.1 s


[(0,
  '0.010*"writing" + 0.007*"text" + 0.007*"item" + 0.007*"word" + 0.006*"discourse" + 0.005*"language" + 0.005*"score" + 0.005*"collaborative" + 0.004*"essay" + 0.004*"sentence"'),
 (1,
  '0.027*"network" + 0.012*"cluster" + 0.010*"forum" + 0.007*"week" + 0.007*"mooc" + 0.005*"participant" + 0.005*"tie" + 0.005*"actor" + 0.005*"moocs" + 0.004*"node"'),
 (2,
  '0.007*"emotion" + 0.007*"grade" + 0.006*"predictive" + 0.006*"score" + 0.004*"assignment" + 0.004*"day" + 0.004*"instructor" + 0.004*"week" + 0.004*"visualization" + 0.004*"prediction"'),
 (3,
  '0.007*"la" + 0.006*"institution" + 0.005*"privacy" + 0.005*"policy" + 0.005*"reflection" + 0.004*"collaborative" + 0.004*"workshop" + 0.003*"video" + 0.003*"pedagogical" + 0.003*"institutional"'),
 (4,
  '0.008*"dialogue" + 0.006*"participant" + 0.005*"message" + 0.004*"cognitive" + 0.004*"reading" + 0.004*"network" + 0.004*"visualisation" + 0.004*"eeg" + 0.004*"utterance" + 0.004*"gesture"'),
 (5,
  '0.005*"condition" + 0.004*"lect

In [12]:
from gensim.models import TfidfModel, LsiModel
%time tfidf_model = TfidfModel(corpus, id2word=dictionary)
%time lsi_model = LsiModel(tfidf_model[corpus], id2word=dictionary, num_topics=10)
lsi_model.print_topics(-1) 

CPU times: user 408 ms, sys: 26.9 ms, total: 435 ms
Wall time: 108 ms
CPU times: user 2.44 s, sys: 50.7 ms, total: 2.49 s
Wall time: 1.84 s


[(0,
  '0.126*"cluster" + 0.125*"network" + 0.115*"mooc" + 0.113*"false" + 0.110*"true" + 0.095*"la" + 0.081*"dashboard" + 0.081*"moocs" + 0.080*"forum" + 0.080*"week"'),
 (1,
  '-0.401*"false" + -0.384*"true" + -0.160*"adobe" + -0.106*"vsamples" + -0.106*"qfactor" + -0.106*"tilewidth" + -0.106*"hsamples" + -0.106*"tileheight" + -0.083*"pdf" + -0.079*"acrobat"'),
 (2,
  '0.313*"privacy" + -0.274*"mooc" + -0.260*"cluster" + 0.243*"policy" + 0.211*"la" + 0.175*"workshop" + 0.166*"institution" + 0.165*"ethical" + -0.150*"moocs" + -0.134*"forum"'),
 (3,
  '0.427*"network" + 0.206*"tie" + 0.174*"node" + 0.170*"centrality" + 0.165*"actor" + 0.147*"forum" + 0.138*"mooc" + -0.102*"bkt" + -0.098*"grade" + 0.096*"networked"'),
 (4,
  '0.412*"mooc" + 0.262*"moocs" + -0.246*"writing" + 0.207*"cluster" + -0.167*"essay" + 0.143*"privacy" + 0.128*"policy" + 0.127*"module" + 0.126*"institution" + -0.125*"discourse"'),
 (5,
  '0.374*"writing" + 0.297*"mooc" + 0.277*"essay" + -0.181*"network" + 0.174*"m